# Semana 11

## Serialización de objetos

Toda la imfo que almacena un compu se guarda en base a bits (1 y 0) y bytes (secuecias de 8 bits).

IMaginemos que buscamos guardar una estructura de datos o una instancia de una clase para volver a leerla más adelante. Esto se llama **serialización.**

Se refiere al procedimiento de transformar un objeto en una **secuncia o serie de bytes**. Esto nos permite almacenar su **info y el estado de un objeto de forma persistente**. Por ejemplo una base da datos para consultarla más adelante
NOs permite enviear el **objeto a través de la red**.(a otros compus y programas)

## `pickle`
El modulo `pickle` de Python nos permite guardar y argar casi cualquier objeto. (serializar objetos).

Ofrece los métodos.

* `dumps`: **Guarda** un objeto en un archivo. Es decir lo **serializa**.
* `loads`: Carga un objeto desde un archivo. Es decir lo **deserializa**.

Una vez un objeto es serializado es persistente y puede volver a ser usado a futuo por el mismo u otro programa.

```python  
import pickle


tupla = ("a", 1, 3, "hola")
tupla_serializada = pickle.dumps(tupla)

print(f"Resultado serialización: {tupla_serializada}")
print(f"Tipo de versión serializada: {type(tupla_serializada)}")

tupla_deserializada = pickle.loads(tupla_serializada)
print(f"Resultado deserialización: {tupla_deserializada}")

print()
print(f"Tupla original: {tupla}")
print(f"Tupla deserializada: {tupla_deserializada}")
print(f"¿Las tuplas son iguales? {tupla == tupla_deserializada}")
print(f"¿Las tuplas son el mismo objeto? {tupla is tupla_deserializada}")

'''
Resultado serialización: b'\x80\x04\x95\x13\x00\x00\x00\x00\x00\x00\x00(\x8c\x01a\x94K\x01K\x03\x8c\x04hola\x94t\x94.'
Tipo de versión serializada: <class 'bytes'>
Resultado deserialización: ('a', 1, 3, 'hola')

Tupla original: ('a', 1, 3, 'hola')
Tupla deserializada: ('a', 1, 3, 'hola')
¿Las tuplas son iguales? True
¿Las tuplas son el mismo objeto? False
'''
```

Además ofrece los métodos muy parecidops a los anteriores pero sin la `s` al final.

Que guardan y deserializan **archivos.**

```python
from os import path

lista = [1, 2, 3, 7, 8, 3]

with open(path.join("data", "mi_lista.bin"), "wb") as file:
    pickle.dump(lista, file)

with open(path.join("data", "mi_lista.bin"), "rb") as file:
    lista_cargada = pickle.load(file)

print(f"Lista original: {lista}")
print(f"Lista cargada : {lista_cargada}")
print(f"¿Las listas son iguales? {lista == lista_cargada}")
print(f"¿Las listas son el mismo objeto? {lista is lista_cargada}")
    
'''
print(f"Lista original: {lista}")
print(f"Lista cargada : {lista_cargada}")
print(f"¿Las listas son iguales? {lista == lista_cargada}")
print(f"¿Las listas son el mismo objeto? {lista is lista_cargada}")
'''
```

Pickle puede ser usado de manera no segura. Si se carga un archivo malicioso, este podría ejecutar código malicioso. Por lo que se recomienda no cargar archivos de fuentes desconocidas. [enlace](https://checkoway.net/musings/pickle/)

### Personalización de la serialización en `pickle`

Cuando se serializa un objeto, `pickle` verifica que el objeto tenga el método `__getstate__`. Este método retorna un diccionario con los atributos que se quieren serializar. Si este metodo no existe entonces, `pickle` guardará el atributo `__dict__` del objeto.

Este atributo es un diccionario quer guarda todos los atributos y ,étodos de un objeto.

En otras palabras, objeto.atrubito es equivalentemente a `objeto.__dict__["atributo"]`
y objeto.atributo = valor es equivalente a `objeto.__dict__["atributo"] = valor`, y `objeto.__dict__['metodo']()` es equivalente a `objeto.metodo()`.

Usando el método __getstate__ podemos personalizar la serialización de un objeto. Haciendo que pickle serialice solo los atributos que queramos.

```python
class Persona:

    def __init__(self, nombre, edad):
        self.nombre = nombre
        self.edad = edad
        self.mensaje = "No pasa nada"

    def __getstate__(self):
        """
        Retorna el estado actual del objeto, para que sea serializado por pickle

        Aquí creamos una copia del diccionario actual, para modificar la copia 
        y no el objeto original
        """
        # Usamos una copia del diccionario original para alterar solo la copia
        nueva = self.__dict__.copy()
        # Modificamos un atributo en el objeto serializado.
        # Sin embargo, el objeto original no ha cambiado.
        nueva.update({"mensaje": "¡Me están serializando!"})
        # Lo que retornemos es lo que será serializado por pickle
        return nueva

original = Persona("Juan", 30)
print(f"Mensaje original: {original.mensaje}")
serializado = pickle.dumps(original)
deserializado = pickle.loads(serializado)

# El objeto original sigue igual
print(f"Mensaje original: {original.mensaje}")
print(f"Mensaje deserializado: {deserializado.mensaje}")

'''
Mensaje original: No pasa nada
Mensaje original: No pasa nada
Mensaje deserializado: ¡Me están serializando!
'''
```

# Personalizar la desilización en `pickle`

Utilizamos el metodo '__setstate__' para personalizar la deserialización de un objeto.


```python
class Persona:

    def __init__(self, nombre, edad):
        self.nombre = nombre
        self.edad = edad
        self.mensaje = "No pasa nada"

    def __getstate__(self):
        nueva = self.__dict__.copy()
        print(f"[__getstate__] Serializando a {nueva['nombre']}")
        nueva.update({"mensaje": "¡Me están serializando!"})
        # Lo que retornemos es lo que será serializado por pickle
        return nueva

    def __setstate__(self, state):
        print("[__setstate__] Objeto recién deserializado, actualizando su estado")
        # Al desarializar modificamos el estado
        state.update({"nombre": f"{state['nombre']} deserializado"})
        self.__dict__ = state


original = Persona("Juan", 30)
print(f"Nombre original: {original.nombre}")
# Al usar pickle.dumps() se ejecuta el método __getstate__
serializado = pickle.dumps(original)

'''
Nombre original: Juan
[__getstate__] Serializando a Juan
'''

print("¡Ejecutar loads → deserializar!")
# Al usar pickle.loads() se ejecuta el método __setstate__
deserializado = pickle.loads(serializado)
print(f"Nombre deserializado: {deserializado.nombre}, y su mensaje: {deserializado.mensaje}")

'''
¡Ejecutar loads → deserializar!
[__setstate__] Objeto recién deserializado, actualizando su estado
Nombre deserializado: Juan deserializado, y su mensaje: ¡Me están serializando!
'''
```

Una applicacion que tienen los metodos, es cuando necesitamos serializar un objeto que contiene un atirbuto que depende de las **condiciones actuales del programa**

Imaginemos que un objeto guarda ingo sobre los usarios conectados s actualmente al programa como la cantidad de usaruios y la info correspondiente a la conexión de cada uno. Cuando guardamos el objeto, deberíamos eliminar estas conexiones, ya que al cargalo en otra instancia del programa no deberiamos poder cxomunicarnos con los usarios de la instancia anterior.  Pata lograr esto usamos el metodo `__getstate__` para eliminar la info de los usuarios conectados.


Similarmente cuando se cargue el mismo objeto, desde el archivo serializado, sera necesario volver a crear las conexiones con las condiciones del programa nuevo, para realizar esto usamos el metodo `__setstate__` para crear las conexiones con las condiciones actuales del programa.

## `json`

La desventaja del modulo `pickle` es que los archivos serializados solo puden ser **desiarilizados** por otros archiuvos en Python. En cambio **JSON** (JavaScript Object Notation) es un formato de texto que puede ser leído por cualquier lenguaje de programación. Se utiliza para traspasar info entre variamos computadors por internet por ejemplo. Ademas es humanamente legible.

Es muy parecido a los diccionarios de Python,

En JSON solo es posible serializar instancias de `int`, `str`, `float`, `dict`, `bool`, `list`, `tuple` y `NoneType`, de acuerdo a esta tabla de transformación que puedes revisar en [este link](https://docs.python.org/3.7/library/json.html#encoders-and-decoders). **Por defecto no es posible serializar funciones o instancias de otras clases.**

En Python, existe un módulo llamado `json` que provee métodos para serializar objetos en el  formato JSON. Este módulo provee una interfaz similar a la de `pickle`, es decir, los métodos `dump`(`s`) y `load`(`s`).

```python
from itertools import count
import json


class Persona:
    id = count()

    def __init__(self, nombre, edad, estado_civil):
        self.nombre = nombre
        self.edad = edad
        self.estado_civil = estado_civil
        self.id_ = next(self.id)

p = Persona("Juan", 35, "Soltero")

json_string = json.dumps(p.__dict__)
print("Datos en formato JSON:", type(json_string), json_string)

json_deserializado = json.loads(json_string)
print("Datos en formato Python:", type(json_deserializado), json_deserializado)

'''
Datos en formato JSON: <class 'str'> {"nombre": "Juan", "edad": 35, "estado_civil": "Soltero", "id_": 0}
Datos en formato Python: <class 'dict'> {'nombre': 'Juan', 'edad': 35, 'estado_civil': 'Soltero', 'id_': 0}
'''
```

### Personalización de la serialización en `json`

Podemos personalizar la transofrmación utilizando un `json.JSONEncoder`, de forma ánaloga a como lo hicimos con `getstate` en `pickle`.

Creamos una clase que ereda de la clase `json.JSONEncoder` y sobreescribimos el método `default`. Este método recibe un objeto y retorna una representación serializable de este.

```python

from datetime import datetime


class PersonaEncoder(json.JSONEncoder):

    def default(self, obj):
        """Serializa en forma personalizada el objeto de tipo Persona"""
        if isinstance(obj, Persona):
            return {
                "Persona_id": obj.id_,
                "nombre": obj.nombre,
                "edad": obj.edad,
                "estado_civil": obj.estado_civil,
                "ano_nacimiento": datetime.now().year - obj.edad,
            }
        # Mantenemos la serialización por defecto para otros tipos
        return super().default(obj)


# Creamos tres instancias
p1 = Persona("Juanita", 37, "Soltera")
p2 = Persona("Jorge", 33, "Casado")
p3 = Persona("Mariela", 24, "Soltera")


json_string = json.dumps(p1.__dict__)

print(json_string)

'''
{"nombre": "Juanita", "edad": 37, "estado_civil": "Soltera", "id_": 1}
'''

# Ahora lo hacemos con nuestra clase serializadora
json_string = json.dumps(p1, cls=PersonaEncoder)
print(json_string)

json_string = json.dumps(p2, cls=PersonaEncoder)
print(json_string)

json_string = json.dumps(p3, cls=PersonaEncoder)
print(json_string)

'''
{"Persona_id": 1, "nombre": "Juanita", "edad": 37, "estado_civil": "Soltera", "ano_nacimiento": 1985}
{"Persona_id": 2, "nombre": "Jorge", "edad": 33, "estado_civil": "Casado", "ano_nacimiento": 1989}
{"Persona_id": 3, "nombre": "Mariela", "edad": 24, "estado_civil": "Soltera", "ano_nacimiento": 1998}
'''
```


### Personalización de la deserialización en `json`

Para personalizar la deserialización en `json` debemos crear una función que reciba un diccionario y retorne un objeto de la clase que queremos deserializar.

Para hacer esto utilizamos el argumento `object_hook` de la función `json.loads`. Este argumento recibe una función que será llamada por cada diccionario que se deserialice. Funciona de manera ánaloga a `__setstate__` en `pickle`.

Tambien se pueda al crear una funcion que sera el parametro de los metodos `json.loads` y `json.load` que reciba un diccionario y retorne un objeto de la clase que queremos deserializar.

```python

def funcion_hook(diccionario):
    return [(key, value) for key, value in diccionario.items()]


json_string = '{"nombre": "Jorge", "edad": 34, "estado_civil": "casado", "puntaje": 90.5}'
datos = json.loads(json_string, object_hook=funcion_hook)

print(datos)

'''
[('nombre', 'Jorge'), ('edad', 34), ('estado_civil', 'casado'), ('puntaje', 90.5)]
'''
```

Es importante recordar que **todo objeto de JSON es convertido a un diccionario**, y luego entregado al `object_hook`. Esto tiene implicancias cuando tenemos un JSON con objetos anidados:

```python
def funcion_hook(diccionario):
    """Esta función transforma un diccionario en un número"""
    print(f"Me llegó el diccionario: {diccionario}")
    valor = 33 if len(diccionario) > 1 else 42
    print(f"Lo transformaré en {valor}\n")
    return valor


json_string = '{"a": {"b": 1, "c": [2, 3, {}], "d": null, "e": {"f": true}}}'
datos = json.loads(json_string, object_hook=funcion_hook)

print(datos)

'''
Me llegó el diccionario: {}
Lo transformaré en 42

Me llegó el diccionario: {'f': True}
Lo transformaré en 42

Me llegó el diccionario: {'b': 1, 'c': [2, 3, 42], 'd': None, 'e': 42}
Lo transformaré en 33

Me llegó el diccionario: {'a': 33}
Lo transformaré en 42

42
'''
```

¿Qué pasó ahí arriba? Lo primero que podemos notar es que el proceso de conversión funciona de adentro hacia afuera, y además que el resultado que entregue `object_hook` se "arrastra" en el proceso.



## Manejo de bytes

### Bytes y Encoding

Tenemos 8 bits, el computador puede tener hasta 2^8 combinaciones distintas de 0 y 1. Los numeros desde 0 al 255 en codigo binario.

Los caracteres se interpreta por el computador como un numero. Por ejemplo la letra 'A' se interpreta como el numero 65 en codigo ASCII.

Con funciones como **chr** y **ord** podemos convertir un numero a un caracter y viceversa.

```python
print(ord('A'))  # 65
print(chr(65))  # A
```

Es comun para representar Bytes usar la notacion hexadecimal. Por ejemplo el byte 65 se representa como 0x41. Si bien ord('A') = 65, tambien es igual a 0x41.

Se suele antwponer el 0x para represnetar a un hexadecimal.
La funcion **hex** convierte un numero a su representacion hexadecimal.

```python
for i in [0, 8, 12, 15, 16, 42, 100, 255]:
    print(f"Decimal: {i}, Hexadecimal: {hex(i)}")

'''
Decimal: 0, Hexadecimal: 0x0
Decimal: 8, Hexadecimal: 0x8
Decimal: 12, Hexadecimal: 0xc
Decimal: 15, Hexadecimal: 0xf
Decimal: 16, Hexadecimal: 0x10
Decimal: 42, Hexadecimal: 0x2a
Decimal: 100, Hexadecimal: 0x64
Decimal: 255, Hexadecimal: 0xff
'''
```

Un **dígito hexadecimal** es uno de los dígitos: 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, A, B, C, D, E, F, que corresponden a un valor desde el 0 al 15 respectivamente.

Utilizaremos **Unicode**.

## El objeto `bytes`.

Los bytes en phyton se resresnetan con el objeto del tipo -bytes-. Este objeto es inmutable, es decir no se puede modificar una vez creado. 

Para crear un objeto byte se anteone una b como los f strings.

```python
# Lo que está entre las comillas es un objeto de tipo bytes
# La notación \x63 indica el valor hexadecimal 63
# Este ejemplo almacena los caracteres c, l, i, c, h, é
caracteres = b"\x63\x6c\x69\x63\x68\xe9"
print(caracteres)
print(type(caracteres))

'''
b'cliché'
<class 'bytes'>
'''
```

El símbolo escape `\x` indica que los siguientes dos caracteres después de la `x` corresponden a un *byte* usando dígitos hexadecimales. Los *bytes* que coinciden con los *bytes* de ASCII son reconocidos inmediatamente, así cuando los tratamos de imprimir aparecen correctamente (`clich`), el resto (`é`) se imprime como hexadecimal. La `b` en la impresión nos recuerda que lo que está a la derecha es un objeto de tipo `bytes`, no un `str`.

Los *bytes* a secas pueden representar cualquier entidad, desde caracteres codificados de un *string* a pixeles de una imagen. Para poder interpretar correctamente los *bytes*, necesitamos conocer la forma en que fueron codificados. Por ejemplo, un patrón binario de 8 *bits* (1 *byte*) puede corresponder a un carácter en particular si lo decodificamos usando la codificación llamada `latin1`, pero puede corresponder a un carácter completamente distinto si lo decodificamos como un carácter de tipo `utf-16`. 

```python
caracteres = b"\x63\x6c\x69\x63\x68\xe9"  # Secuencia de 6 bytes
print(caracteres)

# Con bytes.decode() interpretamos los bytes utilizando la codificación latin-1 para obtener un string
print(caracteres.decode("latin-1"))

# 0x61 y 0x62 son la representación en hexadecimal de los caracteres 'a' y 'b', respectivamente
caracteres = b"\x61\x62"
print(caracteres.decode("ascii"))

# 97 y 98 corresponden al código ASCII (en decimal) de a y b, respectivamente
caracteres = bytes((97, 98))
print(caracteres)

'''
b'clich\xe9'
cliché
ab
b'ab'
'''
```

```python	
# Esto generará un error ya que sólo se pueden usar literales ASCII para la creación de bytes
caracteres = b"áb"
```

El método `decode` retorna un *string* normal (Unicode). Si, por ejemplo, hubiésemos usado otro alfabeto, habríamos obtenido otro *string*.

```python
caracteres = b'\x63\x6c\x69\x63\x68\xe9'
print(caracteres.decode("latin-1"))
print(caracteres.decode("iso8859-5"))
print(caracteres.decode("CP437"))
print(caracteres.decode("utf-16"))

'''
cliché
clichщ
clichΘ
汣捩
'''
```

Para codificar un string a un alfabeto especifico siplemente usamos el metodo encode de la clase string.

```python
caracteres = "estación"
print(caracteres.encode("UTF-8"))  # 8-bit Unicode Transformation Format
print(caracteres.encode("latin-1"))
print(caracteres.encode("CP437"))

# No se puede codificar en ASCII el caracter "ó" ya que no existe dentro
# de los 128 caracteres de ASCII
print(caracteres.encode("ascii"))

'''
b'estaci\xc3\xb3n'
b'estaci\xf3n'
b'estaci\xa2n'

UnicodeEncodeError: 'ascii' codec can't encode character '\xf3' in position 4: ordinal not in range(128)
'''
```
